# Part 1 - Implementing Dehumidification in OpenStudio

To evaluate the impact of water-based dehumidification, a custom HVAC system was configured using OpenStudio. The following workflow outlines the detailed setup and adjustments performed to incorporate humidity control into the building energy model.

### A. Model Initialization

A baseline OpenStudio Model (OSM) file named `CDD-watercoil.osm` was created, representing the building geometry and thermal zoning. This model served as the foundation for integrating humidity control strategies.

### B. Air Loop Configuration

An air loop HVAC system was added with the following components:
- A variable volume supply fan.
- A **Coil:Cooling:Water** for latent and sensible cooling.
- A gas heating coil for reheating.
- A **SetpointManager:SingleZone:Reheat** for temperature control.

Additionally, an **AirTerminal:SingleDuct:Reheat** with electric reheat was connected to the zone.

To control indoor humidity levels, a **SetpointManager:SingleZone:Humidity:Maximum** was placed at the outlet node of the cooling water coil. This manager sets a maximum humidity ratio in the zone to prevent over-humidification.

### C. Plant Loop Configuration

A dedicated plant loop was created to serve the **Coil:Cooling:Water**. The plant loop included:
- A variable speed pump.
- A **Chiller:Electric:EIR** for chilled water production.
- A **SetpointManager:Scheduled** to maintain the chilled water supply temperature.

To allow system bypass during part-load conditions, adiabatic bypass pipes were added for both the chiller and the cooling coil branches.

### D. Model Simulation

The model was simulated using OpenStudio’s standard EnergyPlus engine to verify the system’s functionality and ensure humidity control components were operational.

### E. IDF File Post-Processing

After the simulation, the generated EnergyPlus IDF file was exported from OpenStudio for refinement. The following modifications were made to enhance humidity control:

- The `Controller:WaterCoil` object associated with the cooling coil was updated by changing the `Control Variable` from `Temperature` to `TemperatureAndHumidity`, enabling the controller to respond to both sensible and latent loads.
- The simulation `Timestep` was adjusted from the default value to **1 minute** (i.e., 60 timesteps per hour), ensuring higher resolution in tracking rapid humidity fluctuations.

These enhancements allowed more precise humidity regulation and accurate assessment of the dehumidification system's performance.


# Part 2 - Editing IDF file to include Desiccant Dehumidifier.
- We need to change `Control Variable` of `Controller:WaterCoil` back to `Temperature` so that cooling based Dehumidification **disabled**.
## 1. Copy file from one directorty and added extension to name

In [1]:
import os 
import subprocess
# Change the current working directory
os.chdir(r"C:\Users\Jayedi Aman\OneDrive - University of Missouri\Desktop\OPENSTUDIO\CDD-Desiccant")

In [2]:
import os
import shutil
from pathlib import Path
import glob

def copy_to_dir(dir1, dir2, file_type='idf', extension='copied.'):
    """
    Copies files from `dir1` to `dir2` after adding a custom extension.
    
    Parameters:
        dir1 (str): Source directory containing files to copy.
        dir2 (str): Destination directory where files will be copied.
        file_type (str): File type to filter for copying (default is 'idf').
        extension (str): Extension to add to copied files (default is 'copied.').
        
    Returns:
        None: The function performs the copying task and does not return any value.
    """
    # Change directory to old idf files to work with
    os.chdir(dir1)
    # Getting names of all idf files only
    files = glob.glob("*"+file_type)

    # Loop to edinting name (copied extension) and saves file to new directory
    for file in files:
        # print(file)
        # print("*"*20)
        st1 = file.split(".")[0]
        st2 = st1 + extension + file.split(".")[1]
        # print(st2)
        # print("*"*20)
        new_dir = os.path.join(dir2, st2)
        shutil.copyfile(file, new_dir)

In [3]:
import os
# # Paths

dir1 = r"C:\Users\Jayedi Aman\OneDrive - University of Missouri\Desktop\OPENSTUDIO\CDD-Desiccant"
dir2 = r"C:\Users\Jayedi Aman\OneDrive - University of Missouri\Desktop\OPENSTUDIO\CDD-Desiccant\New folder"
copy_to_dir(dir1, dir2,file_type='idf', extension='_copied.')

## 2. Finding idf file names in the directory

In [4]:
## List of idf in a folder
import os
def idf_file_list(file_path):
    # Import necessary libraries
    import os, glob, shutil
    # Change directory to old idf files to work with
    os.chdir(file_path)

    # Getting names of all idf files only
    files = glob.glob("*.idf")
    return files

file_path = dir2
idf_files = idf_file_list(file_path)
print(idf_files)

['5ZoneBaseboardDehumid_copied.idf', 'CDD-desiccant-airloop_copied.idf', 'CDD-watercoil-dehumid_copied.idf', 'CDD-watercoil-Nodehumid_copied.idf']


## 3. Find the Branch Objects of AirLoopHVAC

We need to Branch of AirLoopHVAC to include Desiccant Dehumidifer Object into main Air Loop.
- Add "Dehumidifier:Desiccant:NoFans as components 11.
- Add "Component 2 Outlet Node" to Desiccant Process Air Inlet Node
- Add "Process Air Outlet Node" to Desiccant object outlet 
- Add "Process Air Outlet Node" to Component 3 (Coil:Cooling:Water) Inlet Node

First, we need to find the Branch object associated with AirLoopHVAC. The following procedure we follow.
- Find the "Branch List Name" from AirLoopHVAC [index = i+6]
- Find the BranchList object named "Branch List Name" 
- Find the "Branch Name" from BranchList [index = i+1]

From the "Branch Name" related to AirLoopHVAC we will get the Branch object. Copy all lines of this object and save as branch_lines.

Then we need to copy fan_outlet_node from Branch. 

In [5]:
idf_file = idf_files[-1]
idf_file

'CDD-watercoil-Nodehumid_copied.idf'

In [6]:
def clean_idf_value(line):
    # Remove comments and trailing punctuation
    return line.split("!")[0].split(",")[0].split(";")[0].strip()

def extract_branch_1_name(file_path):
    with open(file_path, 'r') as f:
        lines = f.readlines()

    # Step 1: Get Branch List Name from AirLoopHVAC object
    for i, line in enumerate(lines):
        if line.strip().startswith("AirLoopHVAC,"):
            branch_list_line = lines[i + 5]
            branch_list_name = clean_idf_value(branch_list_line)
            break
    else:
        raise ValueError("AirLoopHVAC object not found.")

    # Step 2: Get Branch 1 Name from BranchList
    for i, line in enumerate(lines):
        if line.strip().startswith("BranchList,"):
            if i + 1 < len(lines):
                name_line = clean_idf_value(lines[i + 1])
                if name_line == branch_list_name:
                    branch_1_line = lines[i + 2]
                    branch_1_name = clean_idf_value(branch_1_line)
                    return branch_1_name

    raise ValueError("BranchList or Branch 1 Name not found.")

def get_branch_lines(file_path, branch_name):
    with open(file_path, 'r') as f:
        lines = f.readlines()

    branch_lines = []
    found = False
    for i, line in enumerate(lines):
        if line.strip().startswith("Branch,"):
            next_line = clean_idf_value(lines[i + 1]) if i + 1 < len(lines) else ""
            if next_line == branch_name:
                found = True
                branch_lines.append(line)
                j = i + 1
                while j < len(lines):
                    branch_lines.append(lines[j])
                    if ";" in lines[j]:
                        break
                    j += 1
                break

    if not found:
        raise ValueError(f"Branch object with name '{branch_name}' not found.")

    return branch_lines

# Example usage
file_path = idf_file
branch_name = extract_branch_1_name(file_path)
print("Branch 1 Name:", branch_name)

branch_lines = get_branch_lines(file_path, branch_name)

print("\nFull Branch Object:\n")
print("".join(branch_lines))

Branch 1 Name: Air Loop HVAC Main Branch

Full Branch Object:

Branch,
    Air Loop HVAC Main Branch,  !- Name
    ,                        !- Pressure Drop Curve Name
    AirLoopHVAC:OutdoorAirSystem,  !- Component 1 Object Type
    OA System,               !- Component 1 Name
    Sup Inlet Node,          !- Component 1 Inlet Node Name
    Mixwd Air Node,          !- Component 1 Outlet Node Name
    Fan:VariableVolume,      !- Component 2 Object Type
    Var Spd Fan,             !- Component 2 Name
    Mixwd Air Node,          !- Component 2 Inlet Node Name
    Cool Inlet Node,         !- Component 2 Outlet Node Name
    Coil:Cooling:Water,      !- Component 3 Object Type
    CHW Clg Coil,            !- Component 3 Name
    Cool Inlet Node,         !- Component 3 Inlet Node Name
    Cool Outlet Node,        !- Component 3 Outlet Node Name
    Coil:Heating:Fuel,       !- Component 4 Object Type
    Gas Htg Coil,            !- Component 4 Name
    Cool Outlet Node,        !- Component 4

In [7]:
## Collecting node name for Fan Outlet Node
for i in range(len(branch_lines)):
    if "Fan" in branch_lines[i]:
        fan_outlet_node = branch_lines[i + 3].split("!")[0].split(",")[0].strip()
        break

print("Fan outlet node:", fan_outlet_node)

Fan outlet node: Cool Inlet Node


## 4. Include Desiccant Dehumidier as a component to main AirLoopHVAC Branch
- We will add Dehumidifier:Desiccant:NoFans object just before Coil:Cooling:Water
- We need to insert 4 lines-- object, object name, inlet node, outlet node.

We will use the branch_lines copied in earlier section. The list of texts will be used to insert Desiccant objects. Later, we will insert the list texts into IDF file.

In [8]:
# Extracted Fan Node
fan_outlet_node = fan_outlet_node
# Required neDesiccant Object Outlet Node
ProcessAirOutletNode = "Process Air Outlet Node"
# Desiccant Dehumidifier Name
DesiccantDehumidName = "Desiccant 1"

def modify_branch_object(branch_lines):
    """Insert Desiccant before Coil:Cooling:Water and update inlet to Coil dynamically."""
    
    # Build modified lines inserting desiccant with dynamic inlet node
    modified = []
    i = 0
    while i < len(branch_lines):
        line = branch_lines[i]

        if "Coil:Cooling:Water" in line:
            # Insert desiccant block BEFORE this line
            desiccant_block = [
                "    Dehumidifier:Desiccant:NoFans,  !- Component 11 Object Type\n",
                f"    {DesiccantDehumidName},                    !- Component 11 Name\n",
                f"    {fan_outlet_node},              !- Component 11 Inlet Node Name\n",
                f"    {ProcessAirOutletNode},         !- Component 11 Outlet Node Name\n"
            ]
            modified.extend(desiccant_block)

            # Add current and next lines (coil type and coil name)
            modified.append(branch_lines[i])       # Coil:Cooling:Water line
            modified.append(branch_lines[i + 1])   # Coil name line

            # Replace coil inlet node with Process Air Outlet Node
            modified.append(f"    {ProcessAirOutletNode},        !- Component 3 Inlet Node Name\n")
            i += 3  # Skip original coil inlet node line (replaced)

        else:
            modified.append(line)
            i += 1

    return modified

In [9]:
modify_branch_object(branch_lines)

['Branch,\n',
 '    Air Loop HVAC Main Branch,  !- Name\n',
 '    ,                        !- Pressure Drop Curve Name\n',
 '    AirLoopHVAC:OutdoorAirSystem,  !- Component 1 Object Type\n',
 '    OA System,               !- Component 1 Name\n',
 '    Sup Inlet Node,          !- Component 1 Inlet Node Name\n',
 '    Mixwd Air Node,          !- Component 1 Outlet Node Name\n',
 '    Fan:VariableVolume,      !- Component 2 Object Type\n',
 '    Var Spd Fan,             !- Component 2 Name\n',
 '    Mixwd Air Node,          !- Component 2 Inlet Node Name\n',
 '    Cool Inlet Node,         !- Component 2 Outlet Node Name\n',
 '    Dehumidifier:Desiccant:NoFans,  !- Component 11 Object Type\n',
 '    Desiccant 1,                    !- Component 11 Name\n',
 '    Cool Inlet Node,              !- Component 11 Inlet Node Name\n',
 '    Process Air Outlet Node,         !- Component 11 Outlet Node Name\n',
 '    Coil:Cooling:Water,      !- Component 3 Object Type\n',
 '    CHW Clg Coil,         

In [10]:
def update_idf_with_modified_branch(file_path, branch_name, modified_branch_lines):
    with open(file_path, 'r') as f:
        lines = f.readlines()

    start_idx = None
    end_idx = None

    # Find start and end index of the branch object to replace
    for i, line in enumerate(lines):
        if line.strip().startswith("Branch,"):
            next_line = clean_idf_value(lines[i + 1]) if i + 1 < len(lines) else ""
            if next_line == branch_name:
                start_idx = i
                # Find end index by locating the line with semicolon
                for j in range(i, len(lines)):
                    if ';' in lines[j]:
                        end_idx = j
                        break
                break

    if start_idx is None or end_idx is None:
        raise ValueError(f"Branch object '{branch_name}' not found in IDF file.")

    # Replace old branch lines with modified ones
    new_lines = lines[:start_idx] + modified_branch_lines + lines[end_idx + 1:]

    # Write back to file
    with open(file_path, 'w') as f:
        f.writelines(new_lines)

In [11]:
branch_name = extract_branch_1_name(file_path)
branch_lines = get_branch_lines(file_path, branch_name)
modified_branch_lines = modify_branch_object(branch_lines)
update_idf_with_modified_branch(file_path, branch_name, modified_branch_lines)

## 5. Insert Desiccant Dehumidifier, Regeneration Fan and Regeneration Heater objects into IDF file

- Add the fan_outlet_node or "Component 2 Outlet Node" from Branch object to "Process Air Inlet Node" of Dehumidifier:Desiccant:NoFan object

- Add "Regeneration Fan Inlet Node" from Dehumidifier:Desiccant:NoFans to "Air Inlet Node" of Regeneration Fan (Fan:VariableVolume) object
- Add "Air Inlet Node" from "Desiccant Regen Coil" object to "Air Outlet Node" of "Desiccant Regen Fan" object

- Add "Air Outlet Node" of the "Desiccant Regen Fan" (Fan:VariableVolume) to "Air Inlet Node" of this Regeneration heating object (Coil:Heating:Fuel).  
- Add "Process Air Outlet Node" of Dehumidifier:Desiccant as "Air Outlet Node" to heating coil


In [12]:
## Used defined Nodes for new objects
RegenCoilOutNode = "Regen Coil Out Node"
RegenerationFanInletNode = "Regeneration Fan Inlet Node"
RegenFanOutletNode = "Regen Fan Outlet Node"

# Require Regen fan and Regen Heating object name
DesiccantRegenCoilName = "Desiccant Regen Coil"
DesiccantRegenFanName = "Desiccant Regen Fan"

def append_objects_to_idf(file_path, fan_outlet_node):
    # Read the original IDF file lines
    with open(file_path, 'r') as f:
        lines = f.readlines()

    # Prepare the desiccant object block with dynamic inlet node
    desiccant_object = [
        "Dehumidifier:Desiccant:NoFans,\n",
        f"    {DesiccantDehumidName},             !- Name\n",
        "    Always On,               !- Availability Schedule Name\n",
        f"    {fan_outlet_node},           !- Process Air Inlet Node Name\n",
        f"    {ProcessAirOutletNode}, !- Process Air Outlet Node Name\n",
        f"    {RegenCoilOutNode},     !- Regeneration Air Inlet Node Name\n",
        f"    {RegenerationFanInletNode},  !- Regeneration Fan Inlet Node Name\n",
        "    LeavingMaximumHumidityRatioSetpoint,  !- Control Type\n",
        "    0.0074,                    !- Leaving Maximum Humidity Ratio Setpoint {kgWater/kgDryAir}\n",
        "    0.35,                     !- Nominal Process Air Flow Rate {m3/s}\n",
        "    2.5,                     !- Nominal Process Air Velocity {m/s}\n",
        "    10,                      !- Rotor Power {W}\n",
        "    Coil:Heating:Fuel,       !- Regeneration Coil Object Type\n",
        f"    {DesiccantRegenCoilName},    !- Regeneration Coil Name\n",
        "    Fan:VariableVolume,      !- Regeneration Fan Object Type\n",
        f"    {DesiccantRegenFanName},     !- Regeneration Fan Name\n",
        "    DEFAULT;                 !- Performance Model Type\n"
    ]

    fan_block = [
        "Fan:VariableVolume,\n",
        f"    {DesiccantRegenFanName},     !- Name\n",
        "    Always On,               !- Availability Schedule Name\n",
        "    0.7,                     !- Fan Total Efficiency\n",
        "    700.0,                   !- Pressure Rise {Pa}\n",
        "    0.35,                     !- Maximum Flow Rate {m3/s}\n",
        "    FixedFlowRate,           !- Fan Power Minimum Flow Rate Input Method\n",
        "    ,                        !- Fan Power Minimum Flow Fraction\n",
        "    0.0,                     !- Fan Power Minimum Air Flow Rate {m3/s}\n",
        "    0.9,                     !- Motor Efficiency\n",
        "    1.0,                     !- Motor In Airstream Fraction\n",
        "    0,                       !- Fan Power Coefficient 1\n",
        "    1,                       !- Fan Power Coefficient 2\n",
        "    0,                       !- Fan Power Coefficient 3\n",
        "    0,                       !- Fan Power Coefficient 4\n",
        "    0,                       !- Fan Power Coefficient 5\n",
        f"    {RegenerationFanInletNode},  !- Air Inlet Node Name\n",
        f"    {RegenFanOutletNode};   !- Air Outlet Node Name\n"
    ]

    heating_coil_object = [
        "Coil:Heating:Fuel,\n",
        f"     {DesiccantRegenCoilName},    !- Name\n",
        "      Always On,               !- Availability Schedule Name\n",
        "      NaturalGas,              !- Fuel Type\n",
        "      0.80,                    !- Burner Efficiency\n",
        "      40000,                  !- Nominal Capacity {W}\n",
        f"    {RegenFanOutletNode},   !- Air Inlet Node Name\n",
        f"    {RegenCoilOutNode};     !- Air Outlet Node Name\n"
    ]

    # Append desiccant object at the end of the lines
    lines.extend(desiccant_object)
    # Append Regen Heater object at the end of the lines
    lines.extend(heating_coil_object)
    # Append Regen fan object at the end of the lines
    lines.extend(fan_block)

    # Save to the new file
    with open(file_path, "w") as f:
        f.writelines(lines)

    print(f"Modified IDF saved as '{file_path}'")

In [13]:
file_path = file_path
fan_outlet_node = fan_outlet_node  # Your extracted node name here

append_objects_to_idf(file_path, fan_outlet_node)

Modified IDF saved as 'CDD-watercoil-Nodehumid_copied.idf'


## 6. Remove the old OutdoorAir:NodeList
- Find the OutdoorAir:NodeList and look for Nodes' name. 
- Copy the Outdoor Air Inlet Node (e.g OA Inlet Node) and remove OutdoorAir:NodeList

- Create new OutdoorAir:NodeList name it as OutsideAirInletNodes (user defined name)
- Add "Process Air Inlet Node Name" and "Regeneration Fan Inlet Node Name" to this NodeList

The following two Objects will be inserted into IDF file
<pre>
OutdoorAir:NodeList,
    OutsideAirInletNodes;    !- Node or NodeList Name 1

NodeList,
   OutsideAirInletNodes,               !- Name
   OA Inlet Node,                      !- Node 1 Name
   Regeneration Fan Inlet Node;        !- Node 2 Name
</pre>

In [14]:
def list_outdoor_air_nodelists(file_path):
    with open(file_path, 'r') as f:
        lines = f.readlines()

    for i, line in enumerate(lines):
        if line.strip().startswith("OutdoorAir:NodeList,"):
            # Node name is usually the next line, strip comments and whitespace
            node_name_line = lines[i + 1].split("!")[0].strip().rstrip(";")
            print(f"OutdoorAir:NodeList found at line {i + 1}: {node_name_line}")

# Example usage:
list_outdoor_air_nodelists(file_path)

OutdoorAir:NodeList found at line 2594: OA Inlet Node
OutdoorAir:NodeList found at line 2597: Chiller - Air Cooled Inlet Node For Condenser


In [15]:
# Now Find the one OutdoorAir:NodeList which is attached to main AirLoopHVAC

def remove_specific_outdoor_air_nodelist(file_path, node_list_name_to_remove):
    with open(file_path, 'r') as f:
        lines = f.readlines()

    new_lines = []
    skip_block = False
    for i, line in enumerate(lines):
        if line.strip().startswith("OutdoorAir:NodeList,"):
            # Check next line node list name
            next_line = lines[i + 1].split("!")[0].strip().rstrip(";")
            if next_line == node_list_name_to_remove:
                skip_block = True
                continue
        if skip_block:
            if ";" in line:
                skip_block = False
            continue
        new_lines.append(line)

    # Save modified file
    with open(file_path, 'w') as f:
        f.writelines(new_lines)

    print(f"Removed OutdoorAir:NodeList '{node_list_name_to_remove}' and saved as '{file_path}'")

In [16]:
node_list_name = "OA Inlet Node"  # replace with the actual NodeList name to remove

remove_specific_outdoor_air_nodelist(file_path, node_list_name)

Removed OutdoorAir:NodeList 'OA Inlet Node' and saved as 'CDD-watercoil-Nodehumid_copied.idf'


In [17]:
# Required NodeList Name 
OutdoorAirNodeListName = "OutsideAirInletNodes"
# Add Regen Fan Inlet Node to the second Node of OutdoorAir NodeList

def append_new_outdoor_air_nodelist(file_path):
    new_objects = [
        "\n! -- New OutdoorAir:NodeList and NodeList for desiccant system\n",
        "OutdoorAir:NodeList,\n",
        f"    {OutdoorAirNodeListName};    !- Node or NodeList Name 1\n\n",
        "NodeList,\n",
        f"    {OutdoorAirNodeListName},               !- Name\n",
        f"    {node_list_name},                      !- Node 1 Name\n",
        f"    {RegenerationFanInletNode};        !- Node 2 Name\n"
    ]
    with open(file_path, 'a') as f:
        f.writelines(new_objects)
    print(f"Added new OutdoorAir:NodeList and NodeList to '{file_path}'")

In [18]:
append_new_outdoor_air_nodelist(file_path)

Added new OutdoorAir:NodeList and NodeList to 'CDD-watercoil-Nodehumid_copied.idf'


## 7. Replace Coil:Cooling:Water objects' "Air Inlet Node Name"
- Replace "Air Inlet Node" with "Process Air Outlet Node" of Dehumidifier:Desiccant:NoFans object

In [19]:
def update_cooling_coil_air_inlet(file_path, ProcessAirOutletNode):
    with open(file_path, 'r') as f:
        lines = f.readlines()

    for i in range(len(lines)):
        if lines[i].strip().startswith("Coil:Cooling:Water,"):
            target_line_index = i + 12
            if target_line_index < len(lines):
                lines[target_line_index] = f"    {ProcessAirOutletNode},         !- Air Inlet Node Name\n"
            else:
                print("Cannot fine line 12.")
            break
    else:
        print("Coil:Cooling:Water object not found.")

    with open(file_path, 'w') as f:
        f.writelines(lines)

    print("IDF file updated.")

In [20]:
update_cooling_coil_air_inlet(file_path, ProcessAirOutletNode)

IDF file updated.


## 8. Replace SetpointManager:SingleZone:Reheat objects' "Setpoint Node or NodeList Name"
- There are more than one SetpointManager:SingleZone:Reheat object. We will find the one whose "Setpoint Node or NodeList Name" is main AirLoopHVAC Fan's outlet Node (we copied this node as fan_outlet_node).
- Replace this node with "Process Air Outlet Node" of Desiccant object. 

In [21]:
def replace_setpoint_node(file_path, fan_outlet_node, ProcessAirOutletNode):
    with open(file_path, 'r') as f:
        lines = f.readlines()

    new_lines = []
    i = 0
    while i < len(lines):
        line = lines[i]
        new_lines.append(line)

        if line.strip().startswith("SetpointManager:SingleZone:Reheat,"):
            block = [line]
            i += 1
            while i < len(lines):
                block.append(lines[i])
                if ";" in lines[i]:
                    break
                i += 1

            # Check if the last line of the block contains the target node
            if fan_outlet_node in block[-1]:
                block[-1] = f"    {ProcessAirOutletNode};         !- Setpoint Node or NodeList Name\n"

            new_lines = new_lines[:-1] + block  # replace the last added line with full updated block
        i += 1

    with open(file_path, 'w') as f:
        f.writelines(new_lines)

    print(f"Updated setpoint node '{fan_outlet_node}' to '{ProcessAirOutletNode}' in the IDF file.")

In [22]:
replace_setpoint_node(file_path, fan_outlet_node, ProcessAirOutletNode)

Updated setpoint node 'Cool Inlet Node' to 'Process Air Outlet Node' in the IDF file.


## 9. Running the simulation with EnergyPlus

In [23]:
import os
import subprocess

output_directory_list = []

def run_energyplus_single(idf_file, epw_file, eplus_dir):
    base_name = os.path.splitext(os.path.basename(idf_file))[0]
    idf_directory = os.path.dirname(os.path.abspath(idf_file))
    output_directory = os.path.join(idf_directory, f"results_{base_name}")

    if not os.path.exists(output_directory):
        os.makedirs(output_directory)

    output_directory_list.append(output_directory)

    energyplus_exe = os.path.join(eplus_dir, "energyplus.exe")

    cl_st = [
        energyplus_exe,
        "--readvars",
        f"--output-directory={output_directory}",
        f"--weather={epw_file}",
        idf_file,
    ]

    try:
        result = subprocess.run(cl_st, capture_output=True, text=True, check=True)
        print(f"Simulation for {base_name}.idf completed successfully.")
    except subprocess.CalledProcessError as e:
        print(f"Simulation for {base_name}.idf failed.")
        print("Error:", e.stderr)
        print(f"Check log at: {os.path.join(output_directory, 'eplusout.err')}")

# idf_file = "CDD-watercoil-Nodehumid_copied.idf"
idf_file = idf_file

epw_file = r"C:\Users\Jayedi Aman\OneDrive - University of Missouri\Desktop\OPENSTUDIO\weatherfiles\USA_MO_Springfield.Rgnl.AP.724400_TMY3.epw"
eplus_dir = r"C:\EnergyPlusV22-2-0"

run_energyplus_single(idf_file, epw_file, eplus_dir)
print(output_directory_list)

Simulation for CDD-watercoil-Nodehumid_copied.idf completed successfully.
['C:\\Users\\Jayedi Aman\\OneDrive - University of Missouri\\Desktop\\OPENSTUDIO\\CDD-Desiccant\\New folder\\results_CDD-watercoil-Nodehumid_copied']


## 10. Getting results

In [24]:
## Printing the new created folders with simulation results 
print(output_directory_list)

['C:\\Users\\Jayedi Aman\\OneDrive - University of Missouri\\Desktop\\OPENSTUDIO\\CDD-Desiccant\\New folder\\results_CDD-watercoil-Nodehumid_copied']


In [25]:
import os
import glob
import pandas as pd
from bs4 import BeautifulSoup

def htm_file_list(directory):
    """Returns a list of .htm files in the given directory."""
    if not os.path.isdir(directory):
        raise NotADirectoryError(f"{directory} is not a valid directory.")
    return glob.glob(os.path.join(directory, "*.htm"))


def extract_heating_cooling_loads(html_file):
    """Extracts total heating and cooling load from an EnergyPlus HTML report."""
    with open(html_file, 'r', encoding='utf-8') as file:
        soup = BeautifulSoup(file, 'html.parser')

    # Locate "End Uses" table
    end_uses_tag = soup.find('b', string="End Uses")
    if not end_uses_tag:
        return None, None  # No table found, return None values

    target_table = end_uses_tag.find_next('table')
    if not target_table:
        return None, None  # No table found

    # Initialize load values
    heating_load = 0.0
    cooling_load = 0.0
    fan_load = 0.0

    # Process table rows
    rows = target_table.find_all('tr')[1:]  # Skip header row
    for row in rows:
        cols = row.find_all('td')
        end_use = cols[0].text.strip().lower()

        if end_use == "heating":
            heating_load = sum(float(col.text.strip()) for col in cols[1:])
        elif end_use == "cooling":
            cooling_load = sum(float(col.text.strip()) for col in cols[1:])
        elif end_use == "fans":
            fan_load = sum(float(col.text.strip()) for col in cols[1:])


    return heating_load, cooling_load, fan_load

# Initialize DataFrame
df_results = pd.DataFrame(columns=['Folder Name', 'File Name', 'Heating Load (GJ)', 'Cooling Load (GJ)', 'Fan Load (GJ)', 'Total HVAC Load (GJ)'])

# Process all output directories
for folder in output_directory_list:
    folder = os.path.abspath(folder)  # Ensure absolute path
    try:
        htm_files = htm_file_list(folder)
        folder_name = os.path.basename(folder)  # Extract the last directory name

        for htm_file in htm_files:

            heating, cooling, fan = extract_heating_cooling_loads(htm_file)
            total = float(heating) + float(cooling) + float(fan)

            if heating is not None and cooling is not None:
                new_row = pd.DataFrame([[folder_name, os.path.basename(htm_file), heating, cooling, fan, total]], 
                       columns=['Folder Name', 'File Name', 'Heating Load (GJ)', 'Cooling Load (GJ)', 'Fan Load (GJ)', 'Total HVAC Load (GJ)'])

                if not new_row.isna().all().all():  # Check if the new row is not all NaN
                    df_results = pd.concat([df_results, new_row], ignore_index=True)

    except NotADirectoryError as e:
        print(e)

# Save results to a CSV file
output_csv = os.path.join(os.getcwd(), "heating_cooling_loads.csv")
df_results.to_csv(output_csv, index=False)

# Display final DataFrame
print(df_results)

                              Folder Name     File Name  Heating Load (GJ)  \
0  results_CDD-watercoil-Nodehumid_copied  eplustbl.htm            1505.78   

   Cooling Load (GJ)  Fan Load (GJ)  Total HVAC Load (GJ)  
0             579.45         237.28               2322.51  


C:\Users\Jayedi Aman\AppData\Local\Temp\ipykernel_8624\1404601805.py:68: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_results = pd.concat([df_results, new_row], ignore_index=True)


In [26]:
df_results

,Folder Name,File Name,Heating Load (GJ),Cooling Load (GJ),Fan Load (GJ),Total HVAC Load (GJ)
0,results_CDD-watercoil-Nodehumid_copied,eplustbl.htm,1505.78,579.45,237.28,2322.51
